In [1]:
from api import *

In [2]:
get_cpu_data(1)

Risposta: {'status': 'success', 'data': {'resultType': 'vector', 'result': [{'metric': {'__name__': 'node_cpu_seconds_total', 'cpu': '0', 'group': 'brainstorm', 'instance': 'www1.brainstorm.it:9100', 'job': 'node', 'mode': 'user'}, 'value': [1745998757.531, '5429.24']}, {'metric': {'__name__': 'node_cpu_seconds_total', 'cpu': '1', 'group': 'brainstorm', 'instance': 'www1.brainstorm.it:9100', 'job': 'node', 'mode': 'user'}, 'value': [1745998757.531, '5504.7']}, {'metric': {'__name__': 'node_cpu_seconds_total', 'cpu': '2', 'group': 'brainstorm', 'instance': 'www1.brainstorm.it:9100', 'job': 'node', 'mode': 'user'}, 'value': [1745998757.531, '5446.93']}, {'metric': {'__name__': 'node_cpu_seconds_total', 'cpu': '3', 'group': 'brainstorm', 'instance': 'www1.brainstorm.it:9100', 'job': 'node', 'mode': 'user'}, 'value': [1745998757.531, '5561.78']}]}}
